In [2]:
import mujoco
import mediapy as media

In [10]:
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="green_sphere" pos="0 0 -1" size="1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)

with mujoco.Renderer(model) as renderer:
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)

  media.show_image(renderer.render())

ValueError: XML Error: Schema violation: unrecognized attribute: 'qpos'

Element 'geom', line 5
